In [1]:
import os
import ROOT
##### SELECTING GENERATOR FOR SIGNAL SAMPLE
signal_sample="Signal_Sherpa.root"
qcd_sample='Zmumu_Sherpa.root'
average_mode = False
is_CR=True
post_fit = False
if not post_fit:
    type_of_mc = qcd_sample.strip('.root').split('_')[1]
    vbf_scale = 1.0
    qcd_scale = 1.0
else :
    vbf_scale = 1.198
    qcd_scale = 0.924
    type_of_mc = qcd_sample.strip('.root').split('_')[1]+'_AfterFit'

# Configure histograms to use 
import sys
sys.path.append('/Users/diegomac/Documents/HEP/VBF-Analysis/Scripts/')
from histogramHelpers import histogramsHighStatsZmumu,histogramsLowStatsZmumu
histos = histogramsHighStatsZmumu
if is_CR:
    histos = histogramsLowStatsZmumu

Welcome to JupyROOT 6.26/10


In [2]:
Data = {"Data":["Data.root",ROOT.kBlack,0]}   
Signal = {"Signal":[signal_sample,ROOT.kOrange+7,0]} 
Background = {"Wjets":["Wjets.root",ROOT.kGreen,0],"ttbar":["ttbar.root",ROOT.kYellow,0],
              "VV":["VV.root",ROOT.kBlue,0],
              "SingleTop":["singletop.root",ROOT.kCyan,0],
              "QCD Z":[qcd_sample,ROOT.kViolet,0]}


from histogramHelpers import stackPlot

stackPlot(Data,Signal,Background,histos,type_of_mc,vbf_scale,qcd_scale,average=average_mode,after_fit=post_fit,final_state="Z#rightarrow #mu#mu")

HISTOGRAM =  n_bjets
HISTOGRAM =  lepiso
HISTOGRAM =  n_jets_interval
HISTOGRAM =  lep1_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl
Using following bins...  [-2.5 -2.3 -2.1 -1.9 -1.7 -1.5 -1.3 -1.1 -0.9 -0.7 -0.5 -0.3 -0.1  0.1
  0.3  0.5  0.7  0.9  1.1  1.3  1.5  1.7  1.9  2.1  2.3  2.5]
HISTOGRAM =  lep2_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl
Using following bins...  [-2.5 -2.3 -2.1 -1.9 -1.7 -1.5 -1.3 -1.1 -0.9 -0.7 -0.5 -0.3 -0.1  0.1
  0.3  0.5  0.7  0.9  1.1  1.3  1.5  1.7  1.9  2.1  2.3  2.5]
HISTOGRAM =  ljet0_eta_basic_cuts_ptl
Using following bins...  [-7.  -6.5 -6.  -5.5 -5.  -4.5 -4.  -3.5 -3.  -2.8 -2.6 -2.4 -2.2 -2.
 -1.8 -1.6 -1.4 -1.2 -1.  -0.8 -0.6 -0.4 -0.2  0.   0.2  0.4  0.6  0.8
  1.   1.2  1.4  1.6  1.8  2.   2.2  2.4  2.6  2.8  3.   3.5  4.   4.5
  5.   5.5  6.   6.5  7. ]
HISTOGRAM =  ljet1_eta_basic_cuts_ptl
Using following bins...  [-7.  -6.5 -6.  -5.5 -5.  -4.5 -4.  -3.5 -3.  -2.8 -2.6 -2.4 -2

Info in <TCanvas::Print>: pdf file n_bjets_Sherpa.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas2
Info in <TCanvas::Print>: pdf file lepiso_Sherpa.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas2
Info in <TCanvas::Print>: pdf file n_jets_interval_Sherpa.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas2
Info in <TCanvas::Print>: pdf file lep1_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl_Sherpa.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas2
Info in <TCanvas::Print>: pdf file lep2_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl_Sherpa.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas2
Info in <TCanvas::Print>: pdf file ljet0_eta_basic_cuts_ptl_Sherpa.pdf has been created
Warning in <TCanvas::Constructo